In [ ]:
import sqlite3
from datetime import datetime, timedelta

DB_PATH = "csv/data.db"

def init_login_attempts_table():
    """
    Crée la table login_attempts si elle n'existe pas déjà.
    """
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS login_attempts (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                ip_address TEXT,
                timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        """)

def record_attempt(ip_address):
    """
    Enregistre une tentative de connexion pour l'IP donnée.
    """
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute(
            "INSERT INTO login_attempts (ip_address) VALUES (?)",
            (ip_address,)
        )

def is_brute_force(ip_address, max_attempts=10, interval_seconds=10):
    """
    Vérifie si l'IP a dépassé la limite de tentatives dans l'intervalle donné.
   
    :param ip_address: IP du client.
    :param max_attempts: Nombre max de tentatives.
    :param interval_seconds: Fenêtre de temps en secondes.
    :return: True si limite atteinte.
    """
    with sqlite3.connect(DB_PATH) as conn:
        now = datetime.now()
        interval_start = now - timedelta(seconds=interval_seconds)
        cur = conn.execute("""
            SELECT COUNT(*) FROM login_attempts
            WHERE ip_address = ? AND timestamp >= ?
        """, (ip_address, interval_start))
        count = cur.fetchone()[0]
    return count >= max_attempts

def reset_attempts(ip_address):
    """
    Réinitialise les tentatives pour l'IP donnée.
    """
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute("DELETE FROM login_attempts WHERE ip_address = ?", (ip_address,))

# Exemple d’utilisation
if __name__ == "__main__":
    init_login_attempts_table()
    ip = "192.168.1.42"
   
    # Simuler un enregistrement de tentative
    record_attempt(ip)
   
    if is_brute_force(ip):
        print("Trop de tentatives ! Attendez un peu.")
    else:
        print("Tentative autorisée.")